In [57]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [58]:
import tensorflow as tf
import timeit
import os
from six.moves import cPickle
from text_utils import TextLoader
%run text_utils.py
from tensorflow.contrib import rnn
from char_rnn_model import Model

# Define directories, hyperparameter

In [67]:
data_dir  = './'
BATCH_SIZE = 16
SEQ_LEN = 16
encoding='utf-8'
hidden_layer = 256
learning_rate = 0.001 
epochs = 300

# Load data using TextLoader object

In [70]:
textloader = TextLoader(data_dir, BATCH_SIZE, SEQ_LEN, encoding)
X_train = np.asarray(textloader.x_batches[:int(textloader.num_batches*0.7),:,:,:])
y_train = np.asarray(textloader.y_batches[:int(textloader.num_batches*0.7),:,:])
X_val = np.asarray(textloader.x_batches[int(textloader.num_batches*0.7):int(textloader.num_batches*0.8),:,:,:])
y_val = np.asarray(textloader.y_batches[int(textloader.num_batches*0.7):int(textloader.num_batches*0.8),:,:])
X_test = np.asarray(textloader.x_batches[int(textloader.num_batches*0.8):,:,:,:])
y_test = np.asarray(textloader.y_batches[int(textloader.num_batches*0.8):,:,:])

print("Train:",X_train.shape,y_train.shape)
print("Val  :",X_val.shape,y_val.shape)
print("Test :",X_test.shape,y_test.shape)

Train: (12504, 16, 16, 1) (12504, 16, 1)
Val  : (1787, 16, 16, 1) (1787, 16, 1)
Test : (3573, 16, 16, 1) (3573, 16, 1)


# Create your model object

In [71]:
#Place holders
X = tf.placeholder(tf.float32, [BATCH_SIZE, SEQ_LEN, 1])
Y = tf.placeholder(tf.float32, [BATCH_SIZE, 1])

#Define LSTM
#input gate
weights_input_gate = tf.Variable(tf.truncated_normal([1, hidden_layer], stddev=0.05))
weights_input_hidden = tf.Variable(tf.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
bias_input = tf.Variable(tf.zeros([hidden_layer]))
#forget gate
weights_forget_gate = tf.Variable(tf.truncated_normal([1, hidden_layer], stddev=0.05))
weights_forget_hidden = tf.Variable(tf.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
bias_forget = tf.Variable(tf.zeros([hidden_layer]))
#output gate
weights_output_gate = tf.Variable(tf.truncated_normal([1, hidden_layer], stddev=0.05))
weights_output_hidden = tf.Variable(tf.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
bias_output = tf.Variable(tf.zeros([hidden_layer]))
#memory cell
weights_memory_cell = tf.Variable(tf.truncated_normal([1, hidden_layer], stddev=0.05))
weights_memory_cell_hidden = tf.Variable(tf.truncated_normal([hidden_layer, hidden_layer], stddev=0.05))
bias_memory_cell = tf.Variable(tf.zeros([hidden_layer]))

## Output layer weigts
weights_output = tf.Variable(tf.truncated_normal([hidden_layer, 1], stddev=0.05))
bias_output_layer = tf.Variable(tf.zeros([1]))

def LSTM_cell(input, output, state):
    input_gate = tf.sigmoid(tf.matmul(input, weights_input_gate) + tf.matmul(output, weights_input_hidden) + bias_input)    
    forget_gate = tf.sigmoid(tf.matmul(input, weights_forget_gate) + tf.matmul(output, weights_forget_hidden) + bias_forget)    
    output_gate = tf.sigmoid(tf.matmul(input, weights_output_gate) + tf.matmul(output, weights_output_hidden) + bias_output)
    memory_cell = tf.tanh(tf.matmul(input, weights_memory_cell) + tf.matmul(output, weights_memory_cell_hidden) + bias_memory_cell)
    state = state * forget_gate + input_gate * memory_cell 
    output = output_gate * tf.tanh(state)
    return state, output

outputs = []
for i in range(BATCH_SIZE):
    batch_state = np.zeros([1, hidden_layer], dtype=np.float32) 
    batch_output = np.zeros([1, hidden_layer], dtype=np.float32)
    for j in range(SEQ_LEN):
        batch_state, batch_output = LSTM_cell(tf.reshape(X[i][j], (-1, 1)), batch_state, batch_output)
    outputs.append(tf.matmul(batch_output, weights_output) + bias_output_layer)
losses = []
for i in range(len(outputs)):
    losses.append(tf.losses.mean_squared_error(tf.reshape(Y[i], (-1, 1)), outputs[i]))
    
loss = tf.reduce_mean(losses)
gradients = tf.gradients(loss, tf.trainable_variables())
clipped, _ = tf.clip_by_global_norm(gradients, 4)
optimizer = tf.train.AdamOptimizer(learning_rate)
trained_optimizer = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))

# Training

In [ ]:
session = tf.Session()
session.run(tf.global_variables_initializer())
for i in range(epochs):
    traind_scores = []
    epoch_loss = []
    val_epoch_loss = []
    for j in range(X_train.shape[0]):
        X_batch = X_train[j,:,:,:]
        y_batch = y_train[j,:,:]
        o, c, _ = session.run([outputs, loss, trained_optimizer], feed_dict={X:X_batch, Y:y_batch})     
        epoch_loss.append(c)
        traind_scores.append(o)
    for j in range(X_val.shape[0]):
        X_batch = X_val[j,:,:,:]
        y_batch = y_val[j,:,:]
        c = session.run([loss], feed_dict={X:X_batch, Y:y_batch})     
        val_epoch_loss.append(c)
        
    if (i % 10) == 0:
        print('Epoch {}/{}'.format(i, epochs), 'Training loss: {}'.format(np.mean(epoch_loss)), 'Val loss: {}'.format(np.mean(val_epoch_loss)))

Epoch 0/100 Training loss: 91.37164306640625 Val loss: 82.60543060302734


# Generating Text

In [ ]:
preds = []
for j in range(X_test.shape[0]):
    o = session.run([outputs], feed_dict={X:X_test[j,:,:,:]})
    preds.append(o)
preds = np.squeeze(np.asarray(preds)).flatten()

In [ ]:
print()